In [1]:
import pandas as pd # type: ignore
import numpy as np # type: ignore

import sys

# setting path
sys.path.append('..')

from evaluation import get_time_step_fleet
from utils import load_solution

In [2]:
solution = load_solution("../data/solution_example.json")
df = get_time_step_fleet(solution=solution, ts=1)

In [3]:
df.columns

Index(['datacenter_id', 'server_generation', 'server_id', 'action'], dtype='object')

In [4]:
from utils import load_problem_data
demand, datacenters, servers, selling_prices = load_problem_data('../data')

In [5]:
demand_selected = demand[demand['time_step'] == 1]
demand_selected

,time_step,latency_sensitivity,CPU.S1,CPU.S2,CPU.S3,CPU.S4,GPU.S1,GPU.S2,GPU.S3
0,1,high,4000,0,0,0,30,0,0
168,1,medium,6000,0,0,0,10,0,0
336,1,low,10000,0,0,0,10,0,0


In [6]:
datacenters

,datacenter_id,cost_of_energy,latency_sensitivity,slots_capacity
0,DC1,0.25,low,25245
1,DC2,0.35,medium,15300
2,DC3,0.65,high,7020
3,DC4,0.75,high,8280


In [7]:
merged_servers_df = pd.merge(selling_prices, servers, on='server_generation')

# Calculate Total Revenue (assuming selling price represents revenue per time unit over life expectancy)
merged_servers_df['total_revenue'] = merged_servers_df['selling_price'] * merged_servers_df['life_expectancy']

# Calculate Total Costs
merged_servers_df['total_cost'] = (
    merged_servers_df['purchase_price'] +
    (merged_servers_df['average_maintenance_fee'] * merged_servers_df['life_expectancy']) +
    merged_servers_df['cost_of_moving']
)

# Calculate Profitability
merged_servers_df['profitability'] = merged_servers_df['total_revenue'] - merged_servers_df['total_cost']

# Sort by Profitability
most_profitable_servers = merged_servers_df.sort_values(by='profitability', ascending=False)

most_profitable_servers

,server_generation,latency_sensitivity,selling_price,server_type,release_time,purchase_price,slots_size,energy_consumption,capacity,life_expectancy,cost_of_moving,average_maintenance_fee,total_revenue,total_cost,profitability
14,CPU.S1,high,25.0,CPU,"[1,60]",15000,2,400,60,96,1000,288,2400.0,43648,-41248.0
7,CPU.S1,medium,15.0,CPU,"[1,60]",15000,2,400,60,96,1000,288,1440.0,43648,-42208.0
0,CPU.S1,low,10.0,CPU,"[1,60]",15000,2,400,60,96,1000,288,960.0,43648,-42688.0
15,CPU.S2,high,25.0,CPU,"[37,96]",16000,2,460,75,96,1000,308,2400.0,46568,-44168.0
8,CPU.S2,medium,15.0,CPU,"[37,96]",16000,2,460,75,96,1000,308,1440.0,46568,-45128.0
1,CPU.S2,low,10.0,CPU,"[37,96]",16000,2,460,75,96,1000,308,960.0,46568,-45608.0
16,CPU.S3,high,27.5,CPU,"[73,132]",19500,2,800,120,96,1000,375,2640.0,56500,-53860.0
9,CPU.S3,medium,16.5,CPU,"[73,132]",19500,2,800,120,96,1000,375,1584.0,56500,-54916.0
2,CPU.S3,low,11.0,CPU,"[73,132]",19500,2,800,120,96,1000,375,1056.0,56500,-55444.0
17,CPU.S4,high,30.0,CPU,"[109,168]",22000,2,920,160,96,1000,423,2880.0,63608,-60728.0


In [13]:
import uuid


def generate_random_fleet(datacenters=datacenters, servers=servers, num_servers=100, max_time_step=168):
    # Create lists of valid datacenter IDs and server generations
    datacenter_ids = datacenters['datacenter_id'].tolist()
    server_generations = servers['server_generation'].tolist()

    # Generate random data
    random_fleet = pd.DataFrame({
        'datacenter_id': np.random.choice(datacenter_ids, num_servers),
        'server_generation': np.random.choice(server_generations, num_servers),
        'server_id': [str(uuid.uuid4()) for _ in range(num_servers)],
        'time_step_of_purchase': np.random.randint(1, max_time_step + 1, num_servers)
    })

    return random_fleet

In [14]:
fleet = generate_random_fleet()
fleet

,datacenter_id,server_generation,server_id,time_step_of_purchase
0,DC2,GPU.S1,12a5e21f-7c60-43fb-b904-f6c88ff2dae2,157
1,DC1,GPU.S2,7605dc3d-e76c-43ef-8ba9-3a6d0103b395,164
2,DC2,CPU.S3,2e347ff2-745b-4b67-8e13-35cec4b375b6,62
3,DC2,CPU.S3,511fbba2-9deb-4053-9828-0dc1ea139030,55
4,DC3,CPU.S3,255f9106-fbc6-400f-b028-ff9d9e16c65d,114
...,...,...,...,...
95,DC3,GPU.S3,6feb030f-fee8-4162-aa91-c5f9e9396b87,7
96,DC3,GPU.S3,5444cfee-04d6-4d70-b17e-6959ea86de34,53
97,DC4,CPU.S2,d8543265-005d-4f41-a503-778cbde10fac,40
98,DC1,CPU.S1,0ce3072f-0804-4fd4-b167-7af9f64a1bad,104


In [18]:
server_counts = fleet.groupby(['datacenter_id', 'server_generation']).size().reset_index(name="count")
server_counts

,datacenter_id,server_generation,count
0,DC1,CPU.S1,4
1,DC1,CPU.S2,2
2,DC1,CPU.S3,4
3,DC1,CPU.S4,3
4,DC1,GPU.S1,1
5,DC1,GPU.S2,5
6,DC1,GPU.S3,4
7,DC2,CPU.S1,1
8,DC2,CPU.S2,2
9,DC2,CPU.S3,7


In [19]:
fleet.loc[fleet["datacenter_id"] == "DC4"]

,datacenter_id,server_generation,server_id,time_step_of_purchase
7,DC4,CPU.S4,e2f9d3eb-1678-407f-bedf-d866355e5ca9,146
10,DC4,GPU.S2,0c36e0e9-7607-4f56-a5bc-bb2d174126a2,143
13,DC4,CPU.S1,eadf33d7-ced7-414a-969c-a0dfb0f55648,102
14,DC4,GPU.S2,e1ae0bb9-edc0-4246-b5ef-23dfbdf96416,22
15,DC4,GPU.S1,e62f31e3-44ca-4d31-a37b-886e562e8bf8,2
18,DC4,CPU.S3,5875ea0f-e7c3-464a-9021-ab8de854de82,147
19,DC4,CPU.S3,06970303-bdbd-48cc-889d-b88d7c58b937,76
24,DC4,CPU.S1,3fb535f9-a3e9-41f5-a3cc-95cef92cc37f,105
25,DC4,CPU.S2,26600e35-56eb-4573-93a0-8d01ec04c9f9,143
27,DC4,GPU.S2,e2819c53-c2c0-4bd7-9283-2dd16e74c6a0,31


In [20]:
def get_server_ages(fleet, time_step):
    return fleet['time_step_of_purchase'].apply(lambda x: time_step - x)

In [23]:
get_server_ages(fleet, 168)

0      11
1       4
2     106
3     113
4      54
     ... 
95    161
96    115
97    128
98     64
99     34
Name: time_step_of_purchase, Length: 100, dtype: int64